<a href="https://colab.research.google.com/github/hritik880/Datathon/blob/main/Concrete_Classification_Image_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1ZKrbsUlrsTgJSlFMz2EGmTINFCQ8PcjO',
dest_path='content/concrete_crack_data.zip',
unzip=True)

Unzipping...Done.


In [ ]:
import pandas as pd # Data analysis and manipultion tool
import numpy as np # Fundamental package for linear algebra and multidimensional arrays
import tensorflow as tf # Deep Learning Tool
import os # OS module in Python provides a way of using operating system dependent functionality
import cv2 # Library for image processing
from sklearn.model_selection import train_test_split # For splitting the data into train and validation set
from sklearn.metrics import f1_score

In [ ]:
!pip install split-folders

In [ ]:
# !pip install split_folders
import splitfolders
input_folder = "train"
output = "valid" #where you want the split datasets saved. one will be created if none is set

splitfolders.ratio('/content/content/concrete_cracked_images/train', output="output", seed=42, ratio=(.8, .2)) # ratio of split are in order of train/val/test. You can change to whatever you want. For train/val sets only, you could do .75, .25 for example.

Copying files: 28718 files [00:03, 7855.86 files/s]


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2)


test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_gen = train_datagen.flow_from_directory(r'/content/output/train',
                                                    target_size=(224, 224),
                                                    batch_size=32,
                                                    color_mode="rgb",
                                                    shuffle=True,
                                                    class_mode='categorical')


Found 22974 images belonging to 2 classes.


In [ ]:
test_gen = test_datagen.flow_from_directory(r'/content/output/val',
                                                    target_size=(224, 224),
                                                    batch_size=32,
                                                    color_mode="rgb",
                                                    shuffle =False,
                                                    class_mode='categorical')

Found 5744 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.layers import Input,Convolution2D ,GlobalAveragePooling2D,MaxPooling2D ,Dense, Flatten, Dropout
from tensorflow.keras.models import Model , Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

In [ ]:
inception_base = ResNet50(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = inception_base.output
x = GlobalAveragePooling2D()(x)
# add a fully-connected layer
x = Dense(256, activation='relu')(x)
# and a fully connected output/classification layer
predictions = Dense(2, activation='softmax')(x)
# create the full network so we can train on it
inception_transfer = Model(inputs=inception_base.input, outputs=predictions)

In [ ]:
inception_transfer.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [ ]:
output_dir = r'/content/drive/MyDrive/classification_dphi'
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', patience=4, verbose=1,
                                                factor=0.1, min_lr=0.00001)
save_model = ModelCheckpoint(filepath=output_dir + "/resnet_model.h5", monitor='val_loss', verbose=1,
                                 save_best_only=True, save_weights_only=False, mode='min', save_freq='epoch')

In [ ]:
inception_transfer.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_7[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [ ]:
batch_size = 32
history= inception_transfer.fit_generator(train_gen,
                    epochs=5,
                    steps_per_epoch=(train_gen.n // batch_size) ,
                    validation_data = test_gen,
                    validation_steps = (test_gen.n // batch_size),
                    verbose =1 ,
                    callbacks=[learning_rate_reduction, save_model])


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
717/717 [==============================] - 352s 486ms/step - loss: 0.2184 - accuracy: 0.9130 - val_loss: 1.6219 - val_accuracy: 0.5054

Epoch 00001: val_loss improved from inf to 1.62185, saving model to /content/drive/MyDrive/classification_dphi/resnet_model.h5
Epoch 2/5
717/717 [==============================] - 348s 484ms/step - loss: 0.0206 - accuracy: 0.9958 - val_loss: 0.0091 - val_accuracy: 0.9981

Epoch 00002: val_loss improved from 1.62185 to 0.00906, saving model to /content/drive/MyDrive/classification_dphi/resnet_model.h5
Epoch 3/5
717/717 [==============================] - 347s 484ms/step - loss: 0.0124 - accuracy: 0.9973 - val_loss: 0.0057 - val_accuracy: 0.9988

Epoch 00003: val_loss improved from 0.00906 to 0.00570, saving model to /content/drive/MyDrive/classification_dphi/resnet_model.h5
Epoch 4/5
717/717 [==============================] - 345s 482ms/step - loss: 0.0093 - accuracy: 0.9981 - val_loss: 0.0041 - val_accuracy: 0.9990

Epoch 00004: val_loss impro

In [ ]:
labels = (train_gen.class_indices)
labels = dict((v,k) for k,v in labels.items())

In [ ]:
labels

{0: 'Negative', 1: 'Positive'}

In [ ]:
test_image_order = pd.read_csv("/content/content/concrete_cracked_images/Testing_set_concrete_crack.csv")
test_image_order.head()

,filename
0,Image_1.jpg
1,Image_2.jpg
2,Image_3.jpg
3,Image_4.jpg
4,Image_5.jpg


In [ ]:
file_paths = [[fname, '/content/content/concrete_cracked_images/test/' + fname] for fname in test_image_order['filename']]

In [ ]:
# Confirm if number of images is same as number of labels given
if len(test_image_order) == len(file_paths):
  print('Number of image names i.e. ', len(test_image_order), 'matches the number of file paths i.e. ', len(file_paths))
else:
  print('Number of image names does not match the number of filepaths')

Number of image names i.e.  11282 matches the number of file paths i.e.  11282


In [ ]:
test_images = pd.DataFrame(file_paths, columns=['filename', 'filepaths'])
test_images.head()

,filename,filepaths
0,Image_1.jpg,/content/content/concrete_cracked_images/test/...
1,Image_2.jpg,/content/content/concrete_cracked_images/test/...
2,Image_3.jpg,/content/content/concrete_cracked_images/test/...
3,Image_4.jpg,/content/content/concrete_cracked_images/test/...
4,Image_5.jpg,/content/content/concrete_cracked_images/test/...


In [ ]:
final_test_datagen = ImageDataGenerator(
                rescale =1./255)
final_test_gen =  final_test_datagen.flow_from_directory(r'/content/content/concrete_cracked_images/', classes=['test'],
                                                    target_size=(224, 224),
                                                    batch_size=32,
                                                    color_mode="rgb",
                                                    shuffle =False,
                                                    class_mode=None)

Found 11282 images belonging to 1 classes.


In [ ]:
pred = inception_transfer.predict_generator(final_test_gen)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


In [ ]:
pred[0]

array([9.9927825e-01, 7.2175701e-04], dtype=float32)

In [ ]:
predicted_class = np.argmax(pred,axis=1)

In [ ]:
predicted_class

array([0, 1, 1, ..., 1, 0, 1])

In [ ]:
predicted_class

array([0, 1, 1, ..., 1, 0, 1])

In [ ]:
y_hat = [labels[k] for k in predicted_class]

['Negative',
 'Positive',
 'Positive',
 'Negative',
 'Positive',
 'Positive',
 'Negative',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Negative',
 'Positive',
 'Positive',
 'Negative',
 'Negative',
 'Positive',
 'Positive',
 'Negative',
 'Positive',
 'Negative',
 'Positive',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Positive',
 'Positive',
 'Negative',
 'Positive',
 'Negative',
 'Negative',
 'Positive',
 'Positive',
 'Positive',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Positive',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Positive',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Negative',
 'Positive',
 'Positive',
 'Positive',
 'Negative',
 'Positive',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Positive',
 'Positive',
 'Positive',
 'Negative',
 'Positive',

In [ ]:
res = pd.DataFrame({'filename': test_images['filename'], 'prediction': y_hat}) # prediction is nothing but the final predictions of your model on input features of your new unseen test data
res=res['prediction']
res.to_csv("submission.csv", index = False)

# To download the csv file locally
from google.colab import files
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
res

0        Negative
1        Positive
2        Positive
3        Negative
4        Positive
           ...   
11277    Positive
11278    Positive
11279    Positive
11280    Negative
11281    Positive
Name: prediction, Length: 11282, dtype: object